In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

In [240]:
df = pd.read_csv('E:\Projects\AI_playground\datasets\medical_insurance.csv')
df.head()

<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:1: SyntaxWarning: invalid escape sequence '\P'
C:\Users\raman\AppData\Local\Temp\ipykernel_36712\4158633899.py:1: SyntaxWarning: invalid escape sequence '\P'
  df = pd.read_csv('E:\Projects\AI_playground\datasets\medical_insurance.csv')


,person_id,age,sex,region,urban_rural,income,education,marital_status,employment_status,household_size,...,liver_disease,arthritis,mental_health,proc_imaging_count,proc_surgery_count,proc_physio_count,proc_consult_count,proc_lab_count,is_high_risk,had_major_procedure
0,75722,52,Female,North,Suburban,22700.0,Doctorate,Married,Retired,3,...,0,1,0,1,0,2,0,1,0,0
1,80185,79,Female,North,Urban,12800.0,No HS,Married,Employed,3,...,0,1,1,0,0,1,0,1,1,0
2,19865,68,Male,North,Rural,40700.0,HS,Married,Retired,5,...,0,0,1,1,0,2,1,0,1,0
3,76700,15,Male,North,Suburban,15600.0,Some College,Married,Self-employed,5,...,0,0,0,1,0,0,1,0,0,0
4,92992,53,Male,Central,Suburban,89600.0,Doctorate,Married,Self-employed,2,...,0,1,0,2,0,1,1,0,1,0


In [241]:
Q1 = df['annual_medical_cost'].quantile(0.25)
Q3 = df['annual_medical_cost'].quantile(0.75)

IQR = Q3 - Q1

lower_bound = Q1 - 1.5*IQR
upper_bound = Q3 + 1.5*IQR

df = df[(df['annual_medical_cost'] >= lower_bound) & (df['annual_medical_cost'] <= upper_bound)]

In [242]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 93228 entries, 0 to 99999
Data columns (total 54 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   person_id                    93228 non-null  int64  
 1   age                          93228 non-null  int64  
 2   sex                          93228 non-null  object 
 3   region                       93228 non-null  object 
 4   urban_rural                  93228 non-null  object 
 5   income                       93228 non-null  float64
 6   education                    93228 non-null  object 
 7   marital_status               93228 non-null  object 
 8   employment_status            93228 non-null  object 
 9   household_size               93228 non-null  int64  
 10  dependents                   93228 non-null  int64  
 11  bmi                          93228 non-null  float64
 12  smoker                       93228 non-null  object 
 13  alcohol_freq         

In [243]:
columns = df.columns.tolist()
print(columns)

['person_id', 'age', 'sex', 'region', 'urban_rural', 'income', 'education', 'marital_status', 'employment_status', 'household_size', 'dependents', 'bmi', 'smoker', 'alcohol_freq', 'visits_last_year', 'hospitalizations_last_3yrs', 'days_hospitalized_last_3yrs', 'medication_count', 'systolic_bp', 'diastolic_bp', 'ldl', 'hba1c', 'plan_type', 'network_tier', 'deductible', 'copay', 'policy_term_years', 'policy_changes_last_2yrs', 'provider_quality', 'risk_score', 'annual_medical_cost', 'annual_premium', 'monthly_premium', 'claims_count', 'avg_claim_amount', 'total_claims_paid', 'chronic_count', 'hypertension', 'diabetes', 'asthma', 'copd', 'cardiovascular_disease', 'cancer_history', 'kidney_disease', 'liver_disease', 'arthritis', 'mental_health', 'proc_imaging_count', 'proc_surgery_count', 'proc_physio_count', 'proc_consult_count', 'proc_lab_count', 'is_high_risk', 'had_major_procedure']


In [244]:
columns = [
    "age", "bmi", "systolic_bp", "diastolic_bp", "ldl", "hba1c",
    "smoker", "alcohol_freq",
    "chronic_count", "hypertension", "diabetes", "asthma", "copd",
    "cardiovascular_disease", "cancer_history", "kidney_disease",
    "liver_disease", "arthritis", "mental_health",
    "visits_last_year", "hospitalizations_last_3yrs",
    "days_hospitalized_last_3yrs", "medication_count",
    "proc_imaging_count", "proc_surgery_count", "proc_physio_count",
    "proc_consult_count", "proc_lab_count",
    "sex", "region", "urban_rural", "marital_status",
    "household_size", "dependents",
    "income", "education", "employment_status",
    "plan_type", "network_tier", "deductible", "copay",
    "policy_term_years", "policy_changes_last_2yrs", "provider_quality"
]

l = []
for col in columns:
    temp = df[col].dtype
    if temp not in ["float64", "int64"]:
        l.append(col)

    
print(l)

['smoker', 'alcohol_freq', 'sex', 'region', 'urban_rural', 'marital_status', 'education', 'employment_status', 'plan_type', 'network_tier']


In [245]:
cols = ['smoker', 'alcohol_freq', 'sex', 'region', 'urban_rural', 'marital_status', 'education', 'employment_status', 'plan_type', 'network_tier']
for col in cols:
    print(df[col].unique())


['Never' 'Current' 'Former']
[nan 'Weekly' 'Daily' 'Occasional']
['Female' 'Male' 'Other']
['North' 'Central' 'West' 'South' 'East']
['Suburban' 'Urban' 'Rural']
['Married' 'Single' 'Divorced' 'Widowed']
['Doctorate' 'No HS' 'Some College' 'HS' 'Masters' 'Bachelors']
['Retired' 'Employed' 'Self-employed' 'Unemployed']
['PPO' 'POS' 'HMO' 'EPO']
['Bronze' 'Gold' 'Silver' 'Platinum']


In [246]:
df = df.drop(["person_id",
"risk_score",
"claims_count",
"avg_claim_amount",
"total_claims_paid",
"annual_premium",
"monthly_premium",
"is_high_risk",
"had_major_procedure"], axis=1)

In [247]:
df.head()

,age,sex,region,urban_rural,income,education,marital_status,employment_status,household_size,dependents,...,cancer_history,kidney_disease,liver_disease,arthritis,mental_health,proc_imaging_count,proc_surgery_count,proc_physio_count,proc_consult_count,proc_lab_count
0,52,Female,North,Suburban,22700.0,Doctorate,Married,Retired,3,1,...,0,0,0,1,0,1,0,2,0,1
1,79,Female,North,Urban,12800.0,No HS,Married,Employed,3,1,...,0,0,0,1,1,0,0,1,0,1
3,15,Male,North,Suburban,15600.0,Some College,Married,Self-employed,5,3,...,0,0,0,0,0,1,0,0,1,0
4,53,Male,Central,Suburban,89600.0,Doctorate,Married,Self-employed,2,0,...,0,0,0,1,0,2,0,1,1,0
5,63,Female,North,Rural,305000.0,HS,Single,Employed,3,2,...,0,0,0,0,0,0,0,0,0,1


In [248]:
columns = ['smoker', 'alcohol_freq', 'sex', 'region', 'urban_rural', 'marital_status', 'education', 'employment_status', 'plan_type', 'network_tier']
df_encoded = pd.get_dummies(
    df,
    columns=columns,
    drop_first=True
).astype(int)
df_encoded.head()

,age,income,household_size,dependents,bmi,visits_last_year,hospitalizations_last_3yrs,days_hospitalized_last_3yrs,medication_count,systolic_bp,...,education_Some College,employment_status_Retired,employment_status_Self-employed,employment_status_Unemployed,plan_type_HMO,plan_type_POS,plan_type_PPO,network_tier_Gold,network_tier_Platinum,network_tier_Silver
0,52,22700,3,1,27,2,0,0,4,121,...,0,1,0,0,0,0,1,0,0,0
1,79,12800,3,1,26,2,0,0,3,131,...,0,0,0,0,0,1,0,1,0,0
3,15,15600,5,3,31,0,0,0,1,104,...,1,0,1,0,1,0,0,0,0,1
4,53,89600,2,0,30,3,0,0,2,136,...,0,0,1,0,0,1,0,0,1,0
5,63,305000,3,2,20,1,0,0,2,137,...,0,0,0,0,1,0,0,0,0,1


In [249]:
x = df_encoded.drop("annual_medical_cost", axis=1)
y = df_encoded["annual_medical_cost"]

In [250]:
columns = [
    "age", "bmi", "systolic_bp", "diastolic_bp", "ldl", "hba1c",
    "smoker", "alcohol_freq",
    "chronic_count", "hypertension", "diabetes", "asthma", "copd",
    "cardiovascular_disease", "cancer_history", "kidney_disease",
    "liver_disease", "arthritis", "mental_health",
    "visits_last_year", "hospitalizations_last_3yrs",
    "days_hospitalized_last_3yrs", "medication_count",
    "proc_imaging_count", "proc_surgery_count", "proc_physio_count",
    "proc_consult_count", "proc_lab_count",
    "sex", "region", "urban_rural", "marital_status",
    "household_size", "dependents",
    "income", "education", "employment_status",
    "plan_type", "network_tier", "deductible", "copay",
    "policy_term_years", "policy_changes_last_2yrs", "provider_quality"
]

cols = ['smoker', 'alcohol_freq', 'sex', 'region', 'urban_rural', 'marital_status', 'education', 'employment_status', 'plan_type', 'network_tier']

l = []
for temp_col in columns:
    if temp_col not in cols:
        l.append(temp_col)

print(l)
    



['age', 'bmi', 'systolic_bp', 'diastolic_bp', 'ldl', 'hba1c', 'chronic_count', 'hypertension', 'diabetes', 'asthma', 'copd', 'cardiovascular_disease', 'cancer_history', 'kidney_disease', 'liver_disease', 'arthritis', 'mental_health', 'visits_last_year', 'hospitalizations_last_3yrs', 'days_hospitalized_last_3yrs', 'medication_count', 'proc_imaging_count', 'proc_surgery_count', 'proc_physio_count', 'proc_consult_count', 'proc_lab_count', 'household_size', 'dependents', 'income', 'deductible', 'copay', 'policy_term_years', 'policy_changes_last_2yrs', 'provider_quality']


In [251]:
sc = StandardScaler()
x

,age,income,household_size,dependents,bmi,visits_last_year,hospitalizations_last_3yrs,days_hospitalized_last_3yrs,medication_count,systolic_bp,...,education_Some College,employment_status_Retired,employment_status_Self-employed,employment_status_Unemployed,plan_type_HMO,plan_type_POS,plan_type_PPO,network_tier_Gold,network_tier_Platinum,network_tier_Silver
0,52,22700,3,1,27,2,0,0,4,121,...,0,1,0,0,0,0,1,0,0,0
1,79,12800,3,1,26,2,0,0,3,131,...,0,0,0,0,0,1,0,1,0,0
3,15,15600,5,3,31,0,0,0,1,104,...,1,0,1,0,1,0,0,0,0,1
4,53,89600,2,0,30,3,0,0,2,136,...,0,0,1,0,0,1,0,0,1,0
5,63,305000,3,2,20,1,0,0,2,137,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,50,127200,2,0,28,0,0,0,1,115,...,0,0,0,0,0,0,1,0,0,0
99996,42,21600,2,0,27,0,0,0,1,101,...,0,0,0,0,0,0,1,0,0,1
99997,41,81900,1,0,29,7,0,0,1,128,...,0,0,0,1,0,0,1,1,0,0
99998,51,43400,3,2,21,4,0,0,2,110,...,0,0,0,1,0,1,0,0,0,0


In [252]:
columns = ['age', 'bmi', 'systolic_bp', 'diastolic_bp', 'ldl', 'hba1c', 'chronic_count', 'hypertension', 'diabetes', 'asthma', 'copd', 'cardiovascular_disease', 'cancer_history', 'kidney_disease', 'liver_disease', 'arthritis', 'mental_health', 'visits_last_year', 'hospitalizations_last_3yrs', 'days_hospitalized_last_3yrs', 'medication_count', 'proc_imaging_count', 'proc_surgery_count', 'proc_physio_count', 'proc_consult_count', 'proc_lab_count', 'household_size', 'dependents', 'income', 'deductible', 'copay', 'policy_term_years', 'policy_changes_last_2yrs', 'provider_quality']
x_scaled = x.copy()
x_scaled[columns] = sc.fit_transform(x_scaled[columns])
x_scaled

,age,income,household_size,dependents,bmi,visits_last_year,hospitalizations_last_3yrs,days_hospitalized_last_3yrs,medication_count,systolic_bp,...,education_Some College,employment_status_Retired,employment_status_Self-employed,employment_status_Unemployed,plan_type_HMO,plan_type_POS,plan_type_PPO,network_tier_Gold,network_tier_Platinum,network_tier_Silver
0,0.301721,-0.579777,0.529934,0.107411,0.097604,0.078842,-0.286326,-0.251934,2.336371,0.233414,...,0,1,0,0,0,0,1,0,0,0
1,1.996780,-0.790770,0.529934,0.107411,-0.102344,0.078842,-0.286326,-0.251934,1.498462,0.889258,...,0,0,0,0,0,1,0,1,0,0
3,-2.021139,-0.731095,2.390752,2.212019,0.897399,-1.110907,-0.286326,-0.251934,-0.177355,-0.881521,...,1,0,1,0,1,0,0,0,0,1
4,0.364501,0.846026,-0.400475,-0.944893,0.697451,0.673716,-0.286326,-0.251934,0.660554,1.217180,...,0,0,1,0,0,1,0,0,1,0
5,0.992301,5.436727,0.529934,1.159715,-1.302037,-0.516033,-0.286326,-0.251934,0.660554,1.282764,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.176161,1.647374,-0.400475,-0.944893,0.297553,-1.110907,-0.286326,-0.251934,-0.177355,-0.160093,...,0,0,0,0,0,0,1,0,0,0
99996,-0.326079,-0.603221,-0.400475,-0.944893,0.097604,-1.110907,-0.286326,-0.251934,-0.177355,-1.078274,...,0,0,0,0,0,0,1,0,0,1
99997,-0.388859,0.681920,-1.330884,-0.944893,0.497502,3.053215,-0.286326,-0.251934,-0.177355,0.692505,...,0,0,0,1,0,0,1,1,0,0
99998,0.238941,-0.138609,0.529934,1.159715,-1.102088,1.268591,-0.286326,-0.251934,0.660554,-0.488015,...,0,0,0,1,0,1,0,0,0,0


In [253]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=42)